In [1]:
print(sc)

In [2]:
sc.master

'spark://127.0.0.1:7077'

In [3]:
import pandas as pd
import numpy as np

In [4]:
from pyspark.sql import SQLContext
sql = SQLContext(sc)
print(sql)

In [5]:
path='file://home/datastax/projetnosql/pagecounts-20110101-150000.gz'

In [6]:
path='file://home/datastax/projetnosql/pagecounts-20110101-150000.gz'

In [7]:
path='file:///home/datastax/projetnosql/test.csv'

In [8]:
df_pd = pd.read_csv('pagecounts-20110101-150000.gz', sep=' ', compression='gzip', 
                    header=None, engine='c', dtype={0: np.str, 1: np.str, 2: np.int64, 3:np.int64}, 
                    names=['CodeProjet', 'Page', 'NbVues', 'PoidsPage'])

In [9]:
df_pd['Date'] = '2011-01-01'

In [10]:
df_pd.size

5000

In [ ]:
df = sql.createDataFrame(df_pd)

In [13]:
rdd = sc.parallelize([1,2,3])

In [ ]:
rdd.map(lambda x: x*3).collect()